In [2]:
import numpy as np
import pandas as pd
import scipy
from typing import TypeVar,Mapping, Set, Generic, Sequence
from scipy.stats import norm

### Black-Scholes formulas for European Call/Put Pricing
- Underlying asset price
$$S_t = \mu S_t dt + \sigma S_t dz_t$$
- Value of call option
$$dC(S,t)= (\mu S \frac{dC}{dS} + \frac{dC}{dt} + \frac{1}{2} \sigma^2 S^2 \frac{d^2C}{dS^2})dt + \sigma S \frac{dC}{dS} dz$$
- Replicate call option with cash ($x_t$ units) and underlying asset ($y_t$ units)
$$P_t = x_t B_t + y_t S_t$$
$$dP_t = (r x_t B_t + y_t \mu S_t) dt + y_t \sigma S_t d z_t$$
- Match $dC(S,t)$ with $dP_t$
$$y_t = \frac{dC}{dS}$$
$$r x_t B_t = \frac{dC}{dt} + \frac{1}{2} \sigma^2 S^2 \frac{d^2C}{dS^2}$$
- Match $C_t$ with $P_t$
$$r C_t = \frac{dC}{dt} + \frac{1}{2} \sigma^2 S^2 \frac{d^2C}{dS^2} + r \frac{dC}{dS} S_t$$
- Solve PDE with $C(S,T) = \max \{S-K, 0\}$ and $C(0,t) = 0$.
$$C(S,t) = S_t \Phi(d_1) - e^{-r(T-t)} K \Phi(d_2)$$
$$d_1 = \frac{\log(\frac{S_t}{K}) + (r +\sigma^2/2) (T-t)}{\sqrt{T-t}}$$
$$d_2 = d_1 -\sigma \sqrt{T-t}$$

In [3]:
def european_call(S, K, T, r, sigma, t) -> float:   
    # S, spot price
    # K, strike price
    # T, maturity time
    # t, current time
    # r: risk free rate
    # sigma: sd of underlying asset
    
    t_gap = T-t
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * t_gap) / (sigma * np.sqrt(t_gap))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * t_gap) / (sigma * np.sqrt(t_gap))
    call = (S * norm.cdf(d1, 0, 1) - K * np.exp(-r * t_gap) * norm.cdf(d2, 0, 1))
    return call

def european_put(S, K, T, r, sigma, t) -> float:   
    # S, spot price
    # K, strike price
    # T, maturity time
    # t, current time
    # r: risk free rate
    # sigma: sd of underlying asset
    
    t_gap = T-t
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T-t) / (sigma * np.sqrt(T-t))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T-t) / (sigma * np.sqrt(T-t))
    put = (K * np.exp(-r * T-t) * norm.cdf(-d2, 0, 1) - S * norm.cdf(-d1, 0, 1))
    
    return put

In [ ]:
### Standard binary tree/grid-based numerical algorithm for American Option Pricing 